- Image scale normalization
- Image format normalization (.jpg, .png) ✅
- overlay ✅
- blurring ✅
- pixelate ✅
- sharpness --> if pixel val greater, replace
- shifting ✅
- etching ✅
- merging?

In [ ]:
# import dependencies

import matplotlib.image as mimg
import matplotlib.pyplot as plt
import numpy as np
import math
import random

In [ ]:
# Image initialization
def rgba2rgb(img):
    height, width, _ = img.shape
    rgb_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                rgb_img_arr[i, j, k] = img.item((i, j, k))

    return rgb_img_arr


# 0.299R + 0.587G + 0.114B.
def rgb2gray(img):
    height, width, _ = img.shape
    gray_img_arr = np.zeros(shape=(height, width))

    for i in range(height):
        for j in range(width):
            r_gray = img.item((i, j, 0)) * 0.299
            g_gray = img.item((i, j, 1)) * 0.587
            b_gray = img.item((i, j, 2)) * 0.114

            gray_img_arr[i, j] = r_gray + g_gray + b_gray

    return gray_img_arr


def _get_image(path, ignore_alpha = True):
    img = mimg.imread(path)

    if path.endswith('.jpg') or path.endswith('.tif') or path.endswith('.jpeg'):
        return img / 255 
        
    elif path.endswith('.png') and ignore_alpha:
        return rgba2rgb(img)

    return img


img1 = _get_image('Input Images/Spiral.jpg')
img2 = _get_image('Input Images/Oogway_forest.jpg')
img3 = _get_image('Input Images/F.png')
img5 = _get_image('Input Images/Bokeh1.png')
img6 = _get_image('Input Images/Bokeh2.png')
img7 = _get_image('Input Images/Doge.png')
img8 = _get_image('Input Images/Gull.jpeg')

img_j = _get_image('Input Images/Joshua.png', ignore_alpha=False)

In [ ]:
# dependency functions

# overlay dependency
def _avg_pixel_weighted(channel_val1, weight1, channel_val2, weight2) -> int:
    adjusted_weight_total = weight1 + weight2
    return ( (channel_val1 * weight1 + channel_val2 * weight2)/adjusted_weight_total)

# stop width for pixelate
def _find_stop_pixelate(height,  width, grid_size, start_pixel) -> list:
    x_dim, y_dim = start_pixel
    padding = grid_size//2
    _stop = (padding + 1) if grid_size & 1 else padding

    e_h = height % grid_size if (height - x_dim) <= padding else None # second axis
    e_w = width % grid_size if (width - y_dim) <= padding else None # first axis

    empty_stop_height = -padding + e_h if e_h is not None else _stop
    empty_stop_width = -padding + e_w if e_w is not None else _stop      # refactor _stop to above

    return [empty_stop_height, empty_stop_width]

# stop width for blur
def _find_stop_blur(height, width, grid_size, start_pixel) -> list:
    x_dim, y_dim = start_pixel
    padding = grid_size//2
    _stop = (padding + 1) if grid_size & 1 else padding

    if x_dim < height or y_dim < width:
        e_h = height - x_dim if (height - x_dim) <= padding and (x_dim < height) else _stop
        e_w = width - y_dim if (width - y_dim) <= padding and (y_dim < width) else _stop

    if x_dim >= height or y_dim >= width:
        e_h = height - (x_dim - padding) if (height - x_dim) <= padding and (x_dim >= height) else _stop
        e_w = width - (y_dim - padding) if (width - y_dim) <= padding and (y_dim >= width) else _stop

    return [e_h, e_w]

# blur, pixelate dependency
def _avg_color_in_grid(img, grid_size, start_pixel, img_type):

    x_dim, y_dim = start_pixel
    padding = grid_size//2 # no. of pixels on either side of central pixel

    _stop = (padding + 1) if grid_size & 1 else padding  # (less on right side if even)

    r_total, g_total, b_total = 0, 0, 0

    # height, width, _ = img.shape

    # if x_dim < height - _stop and y_dim < width - _stop:
    try:
        r_total, g_total, b_total = 0, 0, 0

        # default grid_size^2 for all pixels not on top or left edge
        total_pixels = grid_size**2 if x_dim > padding and y_dim > padding else 0

        for _i in range(-padding, _stop):
            for _j in range(-padding, _stop):

                if (x_dim + _i >= 0 ) and (y_dim + _j >= 0):

                    r_total += img.item((x_dim + _i, y_dim + _j, 0))
                    g_total += img.item((x_dim + _i, y_dim + _j, 1))
                    b_total += img.item((x_dim + _i, y_dim + _j, 2))

                    # increment by one if pixels on top or left edge
                    total_pixels += 1 and total_pixels != grid_size**2

    # else:
    except IndexError:
        r_total, g_total, b_total = 0, 0, 0
        height, width, _ = img.shape

        if img_type == "BLUR":
            empty_stop_height, empty_stop_width = _find_stop_blur(height, width, grid_size, start_pixel)
        elif img_type == "PIXELATE":
            empty_stop_height, empty_stop_width = _find_stop_pixelate(height, width, grid_size, start_pixel)

        total_pixels = 0
        for _i in range(-padding, empty_stop_height):
            for _j in range(-padding, empty_stop_width):

                r_total += img.item((x_dim + _i, y_dim + _j, 0))
                g_total += img.item((x_dim + _i, y_dim + _j, 1))
                b_total += img.item((x_dim + _i, y_dim + _j, 2))

                total_pixels+=1

    return np.array((r_total, g_total, b_total)) / total_pixels

In [ ]:
# OVERLAY

def overlay(img1, img1_weight, img2, img2_weight):

    max_x_size = max(img1.shape[0], img2.shape[0])
    max_y_size = max(img1.shape[1], img2.shape[1])

    new_img_arr = np.zeros(shape=(max_x_size, max_y_size, 3))

    for i in range(max_x_size):
        for j in range(max_y_size):
            for k in range(3):

                new_img_arr[i, j, k] = _avg_pixel_weighted(img1.item((i, j, k)), img1_weight, img2.item((i, j, k)), img2_weight)  # 4 sec

    return new_img_arr


In [ ]:
# PIXELATE

def pixelate(img, pixelate_amount):

    height, width, _ = img.shape

    padding = pixelate_amount//2

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(padding, height + padding, pixelate_amount):
        for j in range(padding, width + padding, pixelate_amount):
            color = _avg_color_in_grid(img, pixelate_amount, (i, j), "PIXELATE")

            try:
                _stop = (padding + 1) if pixelate_amount & 1 else padding

                for _i in range(-padding, _stop):
                    for _j in range(-padding, _stop):

                        new_img_arr[i + _i, j + _j] = color

            except(IndexError):
                empty_stop_height, empty_stop_width = _find_stop_pixelate(height, width, pixelate_amount, (i, j))

                for _i in range(-padding, empty_stop_height):
                    for _j in range(-padding, empty_stop_width):

                        new_img_arr[i + _i, j + _j] = color

    return new_img_arr


In [ ]:
# PIXELATE SAVE

def pixelate_save(img, pixelate_amount):

    height, width, _ = img.shape
    new_img_arr = np.zeros(shape=(height//pixelate_amount, width//pixelate_amount, 3))
    pixelated_img = pixelate(img, pixelate_amount)

    for i in range(height//pixelate_amount):
        for j in range(width//pixelate_amount):
            for k in range(3):
                new_img_arr[i, j, k] = pixelated_img.item(i*pixelate_amount, j*pixelate_amount, k)


    return new_img_arr


In [ ]:
# BLUR

import concurrent.futures # multiprocessing dependency

THRESHOLD_SIZE = 180 # TODO: adjust values

# Crops main array given [height_offset, width_offset] 
# * Works
# TODO: Optimize  - (nested functions) - else block
def select_chunk(img, offset) -> np.ndarray:
    img_height, img_width, _ = img.shape

    if (img_height - offset[0] >= THRESHOLD_SIZE) and (img_width - offset[1] >= THRESHOLD_SIZE):
        
        cropped_arr = np.zeros(shape=(THRESHOLD_SIZE, THRESHOLD_SIZE, 3))
        
        for _i, i in enumerate(range(offset[0], offset[0] + THRESHOLD_SIZE)):
            for _j, j in enumerate(range(offset[1], offset[1] + THRESHOLD_SIZE)):
                for k in range(3):
                    cropped_arr[_i, _j, k] = img.item((i, j, k))
    else:

        new_img_height = img_height - offset[0] if (img_height - offset[0]) < THRESHOLD_SIZE else THRESHOLD_SIZE
        new_img_width  = img_width  - offset[1] if (img_width  - offset[1]) < THRESHOLD_SIZE else THRESHOLD_SIZE

        cropped_arr = np.zeros(shape=(new_img_height, new_img_width, 3))

        for _i, i in enumerate(range(offset[0], offset[0] + new_img_height)):
            for _j, j in enumerate(range(offset[1], offset[1] + new_img_width)):
                for k in range(3):
                    cropped_arr[_i, _j, k] = img.item((i, j, k))

    return cropped_arr

# Blurs a given 2D array
# * Works
def blur(img, blur_size = 3) -> np.ndarray:
    # blur_size = int(input("Blur size: "))
    # 2 - 18.5s
    # 3 - 24.6s
    # 5 - ~47s
    # 7 - 1m 57.2s
    # 10 - 2m 37.5s

    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):

            new_img_arr[i, j] = _avg_color_in_grid(img, blur_size, (i, j), "BLUR")

    return new_img_arr


# Concatenates 1D array of img chunks
# * Works
def put_together(all_chunks_l, img):

    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i_1d, chunk in enumerate(all_chunks_l):
        _offset = [ i_1d // (width//THRESHOLD_SIZE + 1), i_1d % (width//THRESHOLD_SIZE + 1)] # This is correct

        chunk_h, chunk_w, _ = chunk.shape
        for i in range(chunk_h):
            for j in range(chunk_w):
                for k in range(3):
                    new_img_arr[i + (_offset[0] * THRESHOLD_SIZE), j + (_offset[1] * THRESHOLD_SIZE), k] = chunk.item((i, j, k))

    return new_img_arr

# + This should be the parent function
# Rn: makes a 1D array of all chunks
def concatenate_blur_imgs(img, blur_size = 3):

    # * region Separating Chunks
    height, width, _ = img.shape

    all_chunks = []

    number_of_blurs = (height//THRESHOLD_SIZE + 1, width//THRESHOLD_SIZE + 1) # This is correct since range(0, …)

    for offset_h in range(number_of_blurs[0]):
        for offset_w in range(number_of_blurs[1]):
            all_chunks.append(select_chunk(img, [offset_h * THRESHOLD_SIZE, offset_w * THRESHOLD_SIZE]))
            # all_chunks[offset_h, offset_w] = select_chunk(img, [offset_h * THRESHOLD_SIZE, offset_w * THRESHOLD_SIZE])

    # I now have a 1-D array of each chunk in the pic
    # * endregion



    # * region Multiprocessing 
    all_chunks_blurred = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # results = [executor.submit(blur, chunk) for chunk in all_chunks]
        results = executor.map(blur, all_chunks)

        
        for f in results:
            # time.sleep(2)
            plt.imshow(f)
            all_chunks_blurred.append(f)

    # * endregion



    # * region Concatenation
    return put_together(all_chunks_blurred, img)
    # * endregion


'''
-> -> -> -> -> v
-> -> -> -> -> v
-> -> -> -> -> v
-> -> -> -> -> v
-> -> -> -> -> v
'''

# print(select_chunk(img2, [0, 10*THRESHOLD_SIZE]).shape)
# plt.imshow(concatenate_blur_imgs(img2, 3))
# plt.imshow(put_together( concatenate_blur_imgs(img2, 3) , img2))
plt.imsave('bruhdsc.jpg', concatenate_blur_imgs(img2))
# plt.imshow(concatenate_blur_imgs(img2))
# if __name__ == "__main__":
#     concatenate_blur_imgs(img2)
# plt.imshow(put_together(concatenate_blur_imgs(img2, 20)))

In [42]:
# BLUR MULTIPROCESSING
import concurrent.futures # multiprocessing dependency

THRESHOLD_SIZE = 180 # TODO: adjust values

# + list of offsets?

def put_together(all_chunks_l, img):

    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i_1d, chunk in enumerate(all_chunks_l):
        _offset = [ i_1d // (width//THRESHOLD_SIZE + 1), i_1d % (width//THRESHOLD_SIZE + 1)] # This is correct

        chunk_h, chunk_w, _ = chunk.shape
        for i in range(chunk_h):
            for j in range(chunk_w):
                for k in range(3):
                    new_img_arr[i + (_offset[0] * THRESHOLD_SIZE), j + (_offset[1] * THRESHOLD_SIZE), k] = chunk.item((i, j, k))

    return new_img_arr

def blur_mproc(img, blur_size = 3):

    img_height, img_width, _ = img.shape
    
    all_chunks = []

    # + - Has to be nested - to avoid reference to img
    # TODO: Refactor the HELL out of this - optimize
    def blur_chunk(offset) -> np.ndarray:

        down_dist = img_height - offset[0]
        right_dist = img_width - offset[1]

        # TODO: test perf_counter - if-block faster than calculating new_img_height ? -

        if (down_dist >= THRESHOLD_SIZE) and (right_dist >= THRESHOLD_SIZE):
            
            cropped_arr_blurred = np.zeros(shape=(THRESHOLD_SIZE, THRESHOLD_SIZE, 3))
            
            # Blur one THRESHOLD chunk
            for _i, i in enumerate(range(offset[0], offset[0] + THRESHOLD_SIZE)):
                for _j, j in enumerate(range(offset[1], offset[1] + THRESHOLD_SIZE)):
                    cropped_arr_blurred[_i, _j] = _avg_color_in_grid(img, blur_size, (i, j), "BLUR")
        else:

            # find dimensions of edge chunk
            new_img_height = down_dist if down_dist < THRESHOLD_SIZE else THRESHOLD_SIZE
            new_img_width  = right_dist if right_dist < THRESHOLD_SIZE else THRESHOLD_SIZE

            cropped_arr_blurred = np.zeros(shape=(new_img_height, new_img_width, 3))

            # Blur edge chunk
            for _i, i in enumerate(range(offset[0], offset[0] + new_img_height)):
                for _j, j in enumerate(range(offset[1], offset[1] + new_img_width)):
                    cropped_arr_blurred[_i, _j] = _avg_color_in_grid(img, blur_size, (i, j), "BLUR")

        return cropped_arr_blurred
    

    number_of_blurs = (img_height//THRESHOLD_SIZE + 1, img_width//THRESHOLD_SIZE + 1) # This is correct since range(0, …)

    for offset_h in range(number_of_blurs[0]):
        for offset_w in range(number_of_blurs[1]):
            all_chunks.append(blur_chunk([offset_h * THRESHOLD_SIZE, offset_w * THRESHOLD_SIZE]))
    

    return put_together(all_chunks, img)

plt.imsave('helpme.png', blur_mproc(img2, blur_size=2))
    

In [ ]:
# SHIFT

def shift(img, operation, start_val_increment=1.6, shift_x=10, shift_y=10, start_offset=0):
    # operation = int(input("what operation should be performed? (0:linear, 1:sin, 2:cos, 3:tan, 4:random)  "))
    multiplicant = 0

    start_val = 0

    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                if operation == 4:
                    multiplicant = random.random() * 2 - 1

                new_img_arr[i, j, k] = img.item((
                    ((int(multiplicant * shift_x)) + i + start_offset) %
                    height,
                    ((int(multiplicant * shift_y)) + j + start_offset) %
                    width, k))

        multiplicant = start_val if operation == 0 else math.sin(start_val) if operation == 1 else math.cos(start_val) if operation == 2 else math.tan(start_val) if operation == 3 else 0

        start_val+=start_val_increment # for sin or cos, min and max shift is 0 and 1.5707 (π/2)

    return new_img_arr


In [ ]:
# SHARPEN (SCRAP)
# new idea:
#   new_pixel.channel = pixel.channel + (fine/255) * pixel.channel

def sharpen(img, sharpen_amt):

    blurred_img = blur(img, blur_size=sharpen_amt)
    fine_img = abs(img - blurred_img)

    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))
    fine_imgx3 = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                current_element = fine_img.item((i, j, k))
                if current_element != 0:
                    fine_imgx3[i, j, k] = min(1, current_element * 2) # creates halos

    for i in range(height):
        for j in range(width):
            for k in range(3):
                img_element = img.item((i, j, k))
                fine_img_element = fine_imgx3.item((i, j, k))

                new_img_arr[i, j, k] = fine_img_element if img_element < fine_img_element else img_element
    return new_img_arr


    # return _impose_fine(img, fine_img, threshold)
    # return fine_img
    # return(img + 0)

In [ ]:
# WOOD ETCHER

def etch(img, brightness_ratio = 1):
    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 4))

    for i in range(height):
        for j in range(width):

            # check for transparency
            if img.item((i, j, 3)) < 0.035:
                continue

            # getting grayscale value 
            r_gray = img.item((i, j, 0)) * 0.299 * brightness_ratio
            g_gray = img.item((i, j, 1)) * 0.587 * brightness_ratio
            b_gray = img.item((i, j, 2)) * 0.114 * brightness_ratio

            luminosity = r_gray + g_gray + b_gray

            # if pixel is dark, make it black
            # else make it transparent
            if luminosity <= 0.5:
                new_img_arr[i, j, 0] = 0
                new_img_arr[i, j, 1] = 0
                new_img_arr[i, j, 2] = 0
                new_img_arr[i, j, 3] = 1
            else:
                new_img_arr[i, j, 3] = 0


    return new_img_arr

    

In [ ]:
# plt.imsave("Output Images/Overlay/Overlay 3-1.jpg", overlay(img1, img2))
# plt.imsave("Output Images/Blur/Blur 20.png", blurred_img)
# plt.imsave("Output Images/Sharpen/Doge1.png", sharpen(img7, 15))
# plt.imsave("Output Images/Sharpen/DogeBlur.png", blur(img7,blur_size=15))
# plt.imsave("Output Images/Sharpen/Gull1.png", sharpen(img8, 10))
# plt.imshow( sharpen(img2, 10))
# plt.imsave("Output Images/Blur/Blur 10.png", blur(img1, blur_size=10))
# plt.imsave("Output Images/Blur/Blur 7.png", blur(img2, blur_size=7))
# plt.imsave("Output Images/Blur/Blur 2.png", blur(img2, blur_size=2))
# plt.imsave("Output Images/Pixelate/Pixelate 20.png", pixelate(img1, 20))
# plt.imsave("Output Images/Distort/Random.png", shift(img2, 4))
# plt.imsave("Output Images/Etcher/Joshua 5.png", etch(img_j, brightness_ratio= 1.73), cmap='gray')

In [ ]:
# luminosity method, B&W but bright enough to be seen
# 0.299R + 0.587G + 0.114B.

In [ ]:
width = 1920
i = 11

_offset = [ i // (width//THRESHOLD_SIZE + 1), i % (width//THRESHOLD_SIZE + 1)]
_offset

In [ ]:
import time

def do_something(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'


with concurrent.futures.ProcessPoolExecutor() as executor:
    secs = [5, 4, 3, 2, 1]
    if __name__ == '__main__':
        results = [executor.submit(do_something, sec) for sec in secs]

In [ ]:
print(img2.shape)

In [ ]:
plt.imsave('blur3.png', blur(img3, 20))